In [2]:
import utils as ut
from collections import defaultdict
from sklearn import preprocessing
from sklearn.datasets import fetch_mldata
import operator
import hickle as hkl
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix
import numpy as np

In [3]:
#Constants
MNISTDS="MNIST"
NGDS="NG"

In [4]:
## Function to print the k nearest neighbours
def print_k_nearest_neighbours(idx , k, similarity_matrix, trainLabels, Dataset,  Asc):
    if Dataset==MNISTDS:
            indexes = ut.generateSqIndexes(70000,idx,len(trainLabels))
            
            simDict={}
            for i in indexes:
                simDict[i[0]]=similarity_matrix[i[1]]
            simDict = sorted(simDict.items(), key=operator.itemgetter(1), reverse=not Asc)[:k]

            print "Indices:"
            for i in simDict:
                print i[0]

            print "Categories:"
            catdict=defaultdict(int)
            for i in simDict:
                print trainLabels[i[0]]
                catdict[trainLabels[i[0]]]+=1
            catdict = sorted(catdict.items(), key=operator.itemgetter(1), reverse=True)
            print "Label:", catdict[0][0]
    
    
    elif Dataset==NGDS:
            simListTemp = np.argsort(similarity_matrix.getrow(idx).toarray()[0])
            if(not Asc):
                simListTemp = simListTemp[::-1]
            simList = []
            for i in simListTemp:
                if i<11314:
                    simList.append(i)
                if len(simList)==k:
                    break

            print "Indices:"
            for i in simList:
                print i

            print "Categories:"
            catdict=defaultdict(int)
            for i in simList:
                print trainLabels[i]
                catdict[trainLabels[i]]+=1
            catdict = sorted(catdict.items(), key=operator.itemgetter(1), reverse=True)
            print "Label:", catdict[0][0]
    
    return

# Function to predict the label based on KNN
def pred_knn(idx , k, similarity_matrix, trainLabels,Dataset, Asc): 
    if Dataset==MNISTDS:
        indexes = ut.generateSqIndexes(70000,idx,len(trainLabels))
        simDict={}
        for i in indexes:
            simDict[i[0]]=similarity_matrix[i[1]]
        simDict = sorted(simDict.items(), key=operator.itemgetter(1), reverse=not Asc)[:k]

        catdict=defaultdict(int)
        for i in simDict:
            catdict[trainLabels[i[0]]]+=1
        
    elif Dataset==NGDS:
        simListTemp = np.argsort(similarity_matrix.getrow(idx).toarray()[0])
        if(not Asc):
            simListTemp = simListTemp[::-1]

        simList = []
        for i in simListTemp:
            if i<11314:
                simList.append(i)
            if len(simList)==k:
                break

        catdict=defaultdict(int)
        for i in simList:
            catdict[trainLabels[i]]+=1
    
    
    catdict = sorted(catdict.items(), key=operator.itemgetter(1), reverse=True)
    return catdict[0][0]

In [5]:
# testRange is a tuple which is the range of elements in sim matrix that needs to be tested.
# Asc is true for Euclidean and False for cosine ...
def TestKnn(k,simMatrix,labels,testRange,Dataset,Asc=True):
    predLabels = []
    for i in range(testRange[0],testRange[1]):
        if i%1000 ==0:
            print "Progress :",i,"/",testRange[1]
        predLabels.append(pred_knn(i,k,simMatrix,labels,Dataset,Asc))
        
    return predLabels

# MNIST DATA

In [31]:
mnist = fetch_mldata('MNIST original', data_home="/Users/sasankauppu/Desktop/Data Mining CS6220/")
mnistLabelsTrain = mnist.target[:60000]
mnistLabelsTest = mnist.target[-10000:]
mnist = None

In [32]:
#KNN with cosine matrix
mnistPairwiseC = hkl.load('mnistcosine.hkl')

In [ ]:
#for optimization 
#from scipy.spatial import distance
#import scipy.sparse
#mnistPairwiseC=distance.squareform(mnistPairwiseC)
#mnistPairwiseC=scipy.sparse.csr_matrix(mnistPairwiseC)

In [33]:
predLabels = TestKnn(3,mnistPairwiseC,mnistLabelsTrain,(60000,70000),MNISTDS,True)

print accuracy_score(mnistLabelsTest,predLabels)

Progress : 60000 / 70000
Progress : 61000 / 70000
Progress : 62000 / 70000
Progress : 63000 / 70000
Progress : 64000 / 70000
Progress : 65000 / 70000
Progress : 66000 / 70000
Progress : 67000 / 70000
Progress : 68000 / 70000
Progress : 69000 / 70000
0.9434


In [34]:
mnistPairwiseC = None

In [25]:
#KNN with eucledian matrix
mnistPairwiseE = hkl.load('mnisteuclidean.hkl')

In [27]:
predLabels = TestKnn(5,mnistPairwiseE,mnistLabelsTrain,(60000,70000),MNISTDS, True)

print accuracy_score(mnistLabelsTest,predLabels)

Progress : 60000 / 70000
Progress : 61000 / 70000
Progress : 62000 / 70000
Progress : 63000 / 70000
Progress : 64000 / 70000
Progress : 65000 / 70000
Progress : 66000 / 70000
Progress : 67000 / 70000
Progress : 68000 / 70000
Progress : 69000 / 70000
0.9426


In [30]:
mnistPairwiseE = None

# NG DATA SET

In [6]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [7]:
ngLabelsTrain = newsgroups_train.target
ngLabelsTest = newsgroups_test.target
print len(ngLabelsTest),len(ngLabelsTrain)

7532 11314


In [ ]:
newsgroups_train.

In [9]:
#KNN with cosine matrix
ngPairwiseC = hkl.load('/Users/sasankauppu/DataMining/Assignment1/ngcosineidf.hkl')

In [39]:
predLabels = TestKnn(5,ngPairwiseC,ngLabelsTrain,(11314,18846),NGDS,False)

print accuracy_score(ngLabelsTest,predLabels)

Progress : 12000 / 18846
Progress : 13000 / 18846
Progress : 14000 / 18846
Progress : 15000 / 18846
Progress : 16000 / 18846
Progress : 17000 / 18846
Progress : 18000 / 18846
0.648300584174


In [21]:
ngPairwiseC = None

In [22]:
#KNN with euclidean matrix
ngPairwiseE = hkl.load('ngeuclideanidf.hkl')

In [23]:
predLabels = TestKnn(5,csr_matrix(ngPairwiseE),ngLabelsTrain,(11314,18846),NGDS,True)

print accuracy_score(ngLabelsTest,predLabels)

Progress : 12000 / 18846
Progress : 13000 / 18846
Progress : 14000 / 18846
Progress : 15000 / 18846
Progress : 16000 / 18846
Progress : 17000 / 18846
Progress : 18000 / 18846
0.648167817313


In [24]:
ngPairwiseE = None

In [10]:
print_k_nearest_neighbours(990, 5, ngPairwiseC, ngLabelsTrain, NGDS,  False)

Indices:
990
6763
8560
4771
6647
Categories:
4
4
4
4
4
Label: 4
